In [1]:
import torch
import torch.nn as nn
import pandas as pd

In [2]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.constants import POSITION_COLS, MASS_CLASS_COLS, BASIC_TRAINING_COLS, FORCE_CLASS_COLS, PUCK_SQUARE_DISTANCES, PUCK_ANGLE_FEATURES
from isaac.sanity import class_proportions
from isaac.models import MultiBranchModel, initialise_model
from isaac.training import training_loop
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data_directory = "features_plots/"
model_directory = "models/feature_selection/"

In [4]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

create_directory(data_directory)
create_directory(model_directory)

cuda:0


In [5]:
NORMALISE_DATA = True
BATCH_SIZE = 128
EPOCHS = 100
STEP_SIZE = 1
SEQ_END = 2700

In [6]:
ALL_COLS = BASIC_TRAINING_COLS + PUCK_SQUARE_DISTANCES + PUCK_ANGLE_FEATURES

In [7]:
train_trials = read_dataset("data/train_passive_trials.h5", n_trials=3500, cols=ALL_COLS)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=700, cols=ALL_COLS)

100%|██████████| 700/700 [00:06<00:00, 108.54it/s]


# TRAINING

In [8]:
INPUT_DIM = None    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

In [9]:
labels = ["distances+angles", "distances", "angles"]
stats_dfs = []

for tr_features, label in zip([ALL_COLS, BASIC_TRAINING_COLS+PUCK_SQUARE_DISTANCES, 
                               BASIC_TRAINING_COLS+PUCK_ANGLE_FEATURES], labels):
    
    loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                      class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                      training_columns=tr_features, batch_size=BATCH_SIZE, 
                                      normalise_data=NORMALISE_DATA, device=device, multiclass=True)
        
    network_params = (len(tr_features), HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

    for seed in [0, 1, 2]:
        df = pd.DataFrame(columns=["features", "Epoch", "Mass Loss", "Force Loss", "Mass Train Accuracy",
                                   "Mass Val Accuracy", "Force Train Accuracy", "Force Val Accuracy", "seed"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
        epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                            error, loaders[0], loaders[1], 
                                                                                            EPOCHS, seq_end=SEQ_END,
                                                                                            step_size=STEP_SIZE,
                                                                                            multibranch=True)
        
        torch.save(best_mass_model.state_dict(), "models/feature_selection/best_mass_model_features_%s_seed_%d.pt" % (label, seed))
        torch.save(best_force_model.state_dict(), "models/feature_selection/best_force_model_features_%s_seed_%d.pt" % (label, seed))
    
        train_accuracies = np.array(epoch_accuracies[0])
        val_accuracies = np.array(epoch_accuracies[1]) 
        
        df["Epoch"] = np.arange(EPOCHS)
        df["Mass Loss"] = epoch_losses[:, 0]
        df["Force Loss"] = epoch_losses[:, 1]        
        df["Mass Train Accuracy"] = train_accuracies[:, 0]
        df["Mass Val Accuracy"] = val_accuracies[:, 0]
        df["Force Train Accuracy"] = train_accuracies[:, 1]
        df["Force Val Accuracy"] = val_accuracies[:,1]
        df["features"] = label
        df["seed"] = str(seed)
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"stats.h5", key="stats")

100%|██████████| 700/700 [00:01<00:00, 688.28it/s]
Train_loss: ([0.91219798 0.7857687 ]) Train_acc: ([54.14285714 56.54285714]) Val_acc: ([42.42857143 52.57142857]): 100%|██████████| 100/100 [53:38<00:00, 32.19s/it]
Train_loss: ([0.86209868 0.8139343 ]) Train_acc: ([55.68571429 55.65714286]) Val_acc: ([50.71428571 51.42857143]): 100%|██████████| 100/100 [53:38<00:00, 32.18s/it]
Train_loss: ([0.88342988 0.76163069]) Train_acc: ([54.74285714 59.14285714]) Val_acc: ([40.42857143 51.42857143]): 100%|██████████| 100/100 [53:38<00:00, 32.20s/it]
100%|██████████| 700/700 [00:01<00:00, 693.00it/s]
Train_loss: ([0.90389988 0.8199015 ]) Train_acc: ([53.28571429 55.77142857]) Val_acc: ([50.57142857 54.85714286]): 100%|██████████| 100/100 [53:08<00:00, 31.87s/it]
Train_loss: ([0.71702896 0.80978502]) Train_acc: ([61.4        56.88571429]) Val_acc: ([52.42857143 52.57142857]): 100%|██████████| 100/100 [53:08<00:00, 31.89s/it]
Train_loss: ([0.55943446 0.56633225]) Train_acc: ([63.28571429 62.1142857